In [16]:
import yfinance as yf
import requests
import pandas as pd
from json.decoder import JSONDecodeError
import json
import numpy as np
import requests_cache

In [39]:
companies = pd.read_csv('../Company/constituents.csv')
ticker = companies['Symbol'].to_list()

In [46]:
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite')
session.headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
                   'Accept': 'application/json;charset=utf-8'}

# Remember to change start_date and end_date in DAG
def get_data_for_multiple_stocks(tickers, start_date='2024-01-01',end_date= '2024-04-01', session= session):
    stocks = dict()
    for ticker in tickers:
        data = yf.download(ticker, start=start_date, end=end_date, session=session)
        data.insert(0, 'Ticker', ticker)
        data = data.reset_index()
        data['Prev Close'] = data['Adj Close'].shift(1)
        data['log_return'] = np.log(data['Adj Close'] / data['Prev Close'])
        df1 = data[['Ticker', 'Date', 'log_return']]
        stocks[ticker] = df1
    return stocks

In [47]:
stocks = get_data_for_multiple_stocks(ticker)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [48]:
# data type transform
for i in ticker:
    data = stocks[i]
    data['Date'] = data['Date'].apply(str)
    data['Date'] = pd.to_datetime(data['Date'], infer_datetime_format=True)
    stocks[i] = data

/var/folders/vw/m_748fln0l3f59v46rr8kv4r0000gp/T/ipykernel_19945/626385042.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data['Date'] = pd.to_datetime(data['Date'], infer_datetime_format=True)
/var/folders/vw/m_748fln0l3f59v46rr8kv4r0000gp/T/ipykernel_19945/626385042.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data['Date'] = pd.to_datetime(data['Date'], infer_datetime_format=True)
/var/folders/vw/m_748fln0l3f59v46rr8kv4r0000gp/T/ipykernel_19945/626385042.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed

In [49]:
df = pd.DataFrame()
for (key,value) in stocks.items():
    df1 = pd.DataFrame(value)
    df = pd.concat([df, df1], ignore_index=True)

In [51]:
df.to_csv('stock_price.csv')